In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Reading

In [53]:
import pandas as pd
pd.set_option('display.max_colwidth',100)

In [54]:
Data=pd.read_excel('drive/My Drive/Summer Internship 2020 July/My Test File/Ankit/2020 Dataset/new data/hasoc_2020_de_train_new.xlsx',sep='\t')
Data.dropna(inplace = True)
Data.head()

,tweet_id,text,task1,task2,ID
0,1133388798925189122,Deutsche rothaarige porno reife deutsche frauen porno. Deutsche politessen pornos porno deutsch ...,NOT,NONE,hasoc_2020_de_2684
1,1127134592517980161,RT @NDRinfo: Die deutsche Klimaaktivistin Luisa Neubauer wirft Kanzlerin Merkel wegen ihrer fehl...,NOT,NONE,hasoc_2020_de_1042
2,1128897106171842560,"@ruhrbahn jeden Morgen eine neue „Fahrzeugstörung“, ihr seid einfach nur zum Kotzen",NOT,NONE,hasoc_2020_de_774
3,1123576753199484928,"@Junge_Freiheit Die Inkas hatten sich schon dämlich angestellt, bei den spanischen Eindringlinge...",NOT,NONE,hasoc_2020_de_559
4,1128743783393312768,RT @technosteron: leute die 'boar' schreiben lassen sich bestimmt auch von ihren analphabetische...,HOF,PRFN,hasoc_2020_de_1969


# Data Preprocessing
Stop Word Removal

In [55]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('german')
def remove_sw(txt):
    no_sw =" ".join([a for a in txt.split() if a not in stop])
    return no_sw
Data['text'] = Data['text'].apply(lambda x: remove_sw(x))
Data[['text','task1']].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,task1
0,Deutsche rothaarige porno reife deutsche frauen porno. Deutsche politessen pornos porno deutsch ...,NOT
1,RT @NDRinfo: Die deutsche Klimaaktivistin Luisa Neubauer wirft Kanzlerin Merkel wegen fehlenden ...,NOT
2,"@ruhrbahn Morgen neue „Fahrzeugstörung“, seid einfach Kotzen",NOT
3,"@Junge_Freiheit Die Inkas schon dämlich angestellt, spanischen Eindringlingen, Deutsche toppt",NOT
4,RT @technosteron: leute 'boar' schreiben lassen bestimmt analphabetischen Vater arsch ficken,HOF


Removing Emoji

In [56]:
import re

def deEmojify(txt):
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',txt)
Data['text']=Data['text'].apply(lambda x: deEmojify(x))
Data[['text','task1']].head()

,text,task1
0,Deutsche rothaarige porno reife deutsche frauen porno. Deutsche politessen pornos porno deutsch ...,NOT
1,RT @NDRinfo: Die deutsche Klimaaktivistin Luisa Neubauer wirft Kanzlerin Merkel wegen fehlenden ...,NOT
2,"@ruhrbahn Morgen neue „Fahrzeugstörung“, seid einfach Kotzen",NOT
3,"@Junge_Freiheit Die Inkas schon dämlich angestellt, spanischen Eindringlingen, Deutsche toppt",NOT
4,RT @technosteron: leute 'boar' schreiben lassen bestimmt analphabetischen Vater arsch ficken,HOF


Removing URL

In [57]:
def remove_URL(txt):
    url= re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",txt)
Data['text']=Data['text'].apply(lambda x:remove_URL(x))
Data['text']=Data['text'].str.replace(r'\d+','')           #remove numbers
Data[['text']].head()

,text
0,Deutsche rothaarige porno reife deutsche frauen porno. Deutsche politessen pornos porno deutsch ...
1,RT @NDRinfo: Die deutsche Klimaaktivistin Luisa Neubauer wirft Kanzlerin Merkel wegen fehlenden ...
2,"@ruhrbahn Morgen neue „Fahrzeugstörung“, seid einfach Kotzen"
3,"@Junge_Freiheit Die Inkas schon dämlich angestellt, spanischen Eindringlingen, Deutsche toppt"
4,RT @technosteron: leute 'boar' schreiben lassen bestimmt analphabetischen Vater arsch ficken


# Train Label Encoding

In [58]:
## task into array
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
labelEncode=preprocessing.LabelEncoder()
labelEncode.fit(Data['task1'])
print (labelEncode.classes_)
train_labelEncode=labelEncode.transform(Data['task1'])

['HOF' 'NOT']


# Tokenization & Lemmatization

In [59]:
#lemmatizer and tokenization   (good for noun)
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
Punctuations= "!'$%&()*+,-./:;”<=>?[\\]^_`{|}~\t\n"

wordnet_lemmatizer=WordNetLemmatizer()   #lemmatizer object
tok =nltk.tokenize.TweetTokenizer() #Tokenizer object
def lemma(sentence):
    token_words=tok.tokenize(sentence)    #Tokenization
    token_words
    for word in token_words:
        if word in Punctuations:
            token_words.remove(word)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(wordnet_lemmatizer.lemmatize(word))      #Stemming
        stem_sentence.append(" ")                      #Join words
    return "".join(stem_sentence)

Data['Lemma']=Data['text'].apply(lambda x:lemma(x))
Data[['Lemma']].head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Lemma
0,Deutsche rothaarige porno reife deutsche frau porno Deutsche politessen porno porno deutsch inzets
1,RT @NDRinfo Die deutsche Klimaaktivistin Luisa Neubauer wirft Kanzlerin Merkel wegen fehlenden U...
2,@ruhrbahn Morgen neue „ Fahrzeugstörung “ seid einfach Kotzen
3,@Junge_Freiheit Die Inkas schon dämlich angestellt spanischen Eindringlingen Deutsche toppt
4,RT @technosteron leute boar schreiben lassen bestimmt analphabetischen Vater arsch ficken


# TF-IDF feature vector 

In [60]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf1 = TfidfVectorizer(ngram_range=(1,1),min_df =2,max_df =0.5)                #tfidf object   for 1 gramm        
features1 = tfidf1.fit_transform(Data['Lemma'])  
vector1=features1.toarray()
vector1[1]

array([0., 0., 0., ..., 0., 0., 0.])

# Reading TEST data

In [61]:
tDATA=pd.read_csv('drive/My Drive/Summer Internship 2020 July/My Test File/Ankit/2020 Dataset/test data/german_test_1509.csv')
tDATA.head()

,tweet_id,text,task1,task2,ID
0,1129095874242650112,@derCarsti Boykottieren hört sich besser an. 💙💙,NOT,NONE,hasoc_2020_de_1053
1,1129004308396236800,"RT @ibikus31: Es wird spekuliert, ob Merkel ein Amt in Brüssel übernimmt.\nWäre es so, wie schät...",NOT,NONE,hasoc_2020_de_671
2,1130896929355907080,Hat #Hitler wirklich den Krieg in der Wüste verloren? Nach der #Welt Schlagzeile hat die #Tagess...,NOT,NONE,hasoc_2020_de_2977
3,1132251534329307136,"RT @Beatrix_vStorch: #May tritt in UK unter Tränen zurück. Wenn #Merkel zurücktritt, dann auch u...",NOT,NONE,hasoc_2020_de_1746
4,1124941869115498496,@justmeDoro Eher nicht. Das Gänse hauen wieder ab in ihre warmen Überwinterungsquartiere. 😅😂,NOT,NONE,hasoc_2020_de_2416


# Test data preprocessing

In [62]:
tDATA['text']=tDATA['text'].apply(lambda x:remove_sw(x))
tDATA['text']=tDATA['text'].apply(lambda x: deEmojify(x))
tDATA['text']=tDATA['text'].str.replace(r'\d+','') 
#tDATA.head()
tDATA['Lemma']=tDATA['text'].apply(lambda x:lemma(x))

# TF-IDF vectorizer for Test Data

In [63]:
t_features1 = tfidf1.transform(tDATA['Lemma'])  
t_vector1=t_features1.toarray()
t_vector1.shape

(526, 1852)

# Test label encoding

In [64]:
test_labelEncode=labelEncode.transform(tDATA['task1'])

#RANDOM FOREST

In [65]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100,max_depth=None,min_samples_split=2, random_state=0)

# Model Training

In [66]:
rf.fit(vector1,train_labelEncode)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

# Model prediction

In [67]:
pred_rf = rf.predict(t_vector1)

# Classification Report

In [68]:
print("Accuracy score =", accuracy_score(test_labelEncode, pred_rf))
print(metrics.classification_report(test_labelEncode, pred_rf))

Accuracy score = 0.8403041825095057
              precision    recall  f1-score   support

           0       0.71      0.63      0.67       134
           1       0.88      0.91      0.90       392

    accuracy                           0.84       526
   macro avg       0.79      0.77      0.78       526
weighted avg       0.84      0.84      0.84       526



# Adding predicted lable as a colum to dataFrame 'tDATA'

In [69]:
tDATA['Predictionlabel']=labelEncode.inverse_transform(pred_rf)
tDATA.head()

,tweet_id,text,task1,task2,ID,Lemma,Predictionlabel
0,1129095874242650112,@derCarsti Boykottieren hört besser an.,NOT,NONE,hasoc_2020_de_1053,@derCarsti Boykottieren hört besser an,NOT
1,1129004308396236800,"RT @ibikus: Es spekuliert, Merkel Amt Brüssel übernimmt. Wäre so, schätzen Sie Zukunft Mitglieds...",NOT,NONE,hasoc_2020_de_671,RT @ibikus Es spekuliert Merkel Amt Brüssel übernimmt Wäre so schätzen Sie Zukunft Mitgliedstaat...,NOT
2,1130896929355907080,Hat #Hitler wirklich Krieg Wüste verloren? Nach #Welt Schlagzeile #Tagesschau reagiert.,NOT,NONE,hasoc_2020_de_2977,Hat #Hitler wirklich Krieg Wüste verloren Nach #Welt Schlagzeile #Tagesschau reagiert,NOT
3,1132251534329307136,"RT @Beatrix_vStorch: #May tritt UK Tränen zurück. Wenn #Merkel zurücktritt, Tränen – Freudenträn...",NOT,NONE,hasoc_2020_de_1746,RT @Beatrix_vStorch #May tritt UK Tränen zurück Wenn #Merkel zurücktritt Tränen – Freudentränen ...,NOT
4,1124941869115498496,@justmeDoro Eher nicht. Das Gänse hauen ab warmen Überwinterungsquartiere.,NOT,NONE,hasoc_2020_de_2416,@justmeDoro Eher nicht Das Gänse hauen ab warmen Überwinterungsquartiere,NOT


# Generating CSV file

In [70]:
GermanTask1File=tDATA[['tweet_id','Predictionlabel','ID']]
GermanTask1File.head()
GermanTask1File.to_csv('submission_DE_A.csv',index=False)